In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing import image
import numpy as np
import seaborn as sns
from tkinter import Tk, filedialog
import os

***PROCESAMIENTO DE DATOS***

In [2]:
DATA_DIR = "data/CatvsDog"  # <-- CAMBIA ESTO a tu ruta local

BATCH_SIZE = 32
IMG_SIZE = (150, 150)

train_dir = os.path.join(DATA_DIR, 'train')
val_dir = os.path.join(DATA_DIR, 'validation')

ds_train = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='binary',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

ds_val = image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='binary',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 17526 files belonging to 2 classes.
Found 2290 files belonging to 2 classes.


***NORMALIZAR IMAGENES***

In [3]:
normalization_layer = layers.Rescaling(1./255)
ds_train = ds_train.map(lambda x, y: (normalization_layer(x), y))
ds_val = ds_val.map(lambda x, y: (normalization_layer(x), y))

# Prefetch
AUTOTUNE = tf.data.AUTOTUNE
ds_train = ds_train.cache().prefetch(buffer_size=AUTOTUNE)
ds_val = ds_val.cache().prefetch(buffer_size=AUTOTUNE)

***MODELO***

In [4]:
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

model = models.Sequential([
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.BatchNormalization(),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


d:\Matias\py\mi_entorno\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


***ENTRENAMIENTO***

In [5]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    ds_train,
    epochs=20,
    validation_data=ds_val,
    callbacks=[early_stopping]
)

Epoch 1/20
548/548 ━━━━━━━━━━━━━━━━━━━━ 220s 399ms/step - accuracy: 0.6590 - loss: 0.6605 - val_accuracy: 0.5611 - val_loss: 0.8289
Epoch 2/20
548/548 ━━━━━━━━━━━━━━━━━━━━ 115s 209ms/step - accuracy: 0.7224 - loss: 0.5768 - val_accuracy: 0.5323 - val_loss: 1.0394
Epoch 3/20
548/548 ━━━━━━━━━━━━━━━━━━━━ 107s 195ms/step - accuracy: 0.7211 - loss: 0.5797 - val_accuracy: 0.5323 - val_loss: 1.0609
Epoch 4/20
548/548 ━━━━━━━━━━━━━━━━━━━━ 126s 230ms/step - accuracy: 0.7210 - loss: 0.5828 - val_accuracy: 0.4699 - val_loss: 1.8699
Epoch 5/20
548/548 ━━━━━━━━━━━━━━━━━━━━ 152s 247ms/step - accuracy: 0.7169 - loss: 0.5859 - val_accuracy: 0.5764 - val_loss: 0.7636
Epoch 6/20
548/548 ━━━━━━━━━━━━━━━━━━━━ 184s 336ms/step - accuracy: 0.7266 - loss: 0.5650 - val_accuracy: 0.5323 - val_loss: 0.7176
Epoch 7/20
548/548 ━━━━━━━━━━━━━━━━━━━━ 107s 194ms/step - accuracy: 0.7149 - loss: 0.5968 - val_accuracy: 0.5310 - val_loss: 0.8791
Epoch 8/20
548/548 ━━━━━━━━━━━━━━━━━━━━ 81s 148ms/step - accuracy: 0.7147 - 

In [6]:
model.save('cats_vs_dogs_model.h5')

***EVALUACION***

In [8]:

loss, accuracy = model.evaluate(ds_val)
print(f'Validation Loss: {loss:.4f}')
print(f'Validation Accuracy: {accuracy:.4f}')

y_true = []
y_pred = []

for images, labels in ds_val:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.round(preds).flatten())

print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=['Cat', 'Dog']))

cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Cat', 'Dog'], yticklabels=['Cat', 'Dog'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

NameError: name 'ds_val' is not defined

***PREDICCION IMAGEN***

In [ ]:
print("\nSelecciona una imagen para predecir...")
root = Tk()
root.withdraw()
img_path = filedialog.askopenfilename(
    title='Seleccionar imagen',
    filetypes=[('Image Files', '*.jpg *.jpeg *.png')]
)

if img_path:
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)

    if prediction < 0.5:
        print(f'La imagen seleccionada es un Gato')
    else:
        print(f'La imagen seleccionada es un Perro')

    img = image.load_img(img_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title("Gato" if prediction < 0.5 else "Perro")
    plt.show()
else:
    print("No se seleccionó ninguna imagen.")

NameError: name 'tfds' is not defined